# Timbermafia styles and palettes

*NOTE: Github's Jupyter notebook rendering will not display output text with the
      appropriate colours. Run this notebook locally for best effects.*

In this notebook we will first demonstrate the pre-packaged styles and palettes.

We will then describe what a style and palette can do in timbermafia, and create custom

versions of each of these and show how to have timbermafia use them in its configuration.

## Preset styles

The preset styles can be seen with the following:

In [1]:
import logging
log = logging.getLogger('mylog')

import timbermafia as tm
tm.print_styles()

- Preset styles:
       default - Default style for timbermafia.
    minimalist - Display only the time and message, good for verbose log messages.
       compact - Give lots of log record information in a small space.
          boxy - A detailed, boxy looking output fit to the terminal.
         plain - A style emulating vanilla logging.


Now let's print out some example output with each style.

Note that the "plain" style does not leverage many of the timbermafia features, which can be 

useful in configuring a custom style from a very bare-bones preset (see later), or just comparing

how more simple logging solutions perform.

In [2]:
long_message = ('Here is a very long message that probably will not fit on a single line. We will use '
                'it to demonstrate what multi-line output looks like in each style.')
                
def print_samples():
    log.info('INFO level output looks like this.')
    log.debug('DEBUG level output looks like this.')
    log.warning('WARNING level output looks like this.')
    log.error('ERROR level output looks like this.')
    log.fatal('CRITICAL level output looks like this.')
    log.info(long_message)
    
class SomeClass(tm.Logged):
    """A class to demonstrate logging from mixin loggers, which provide context
    on what class and method called the output."""
    def status(self):
        log.info('Some output from method "status" in class "SomeClass".')
    def method_with_a_very_very_long_name(self):
        log.info('Output from a function with a long name to demonstrate truncation or multiline, depending on the style.')

s = SomeClass()
def demo():
    print_samples()
    s.status()
    s.method_with_a_very_very_long_name()
        
# Iterate over each style, configure timbermafia with that style, and print output.
for my_style in tm.styles.style_map:
    print(f'- Using style: {my_style}\n')
    # Clear the configured handlers each time, and disable fit_to_terminal in each style
    # because Jupyter notebooks don't work well with this feature.
    tm.basic_config(style=my_style, clear=True, silent=True, fit_to_terminal=False)
    demo()
    print()
    print('*'*100)

- Using style: default

08:54:47 |     INFO |  mylog.print_samples >> INFO level output looks like this.                    
08:54:47 |    DEBUG |  mylog.print_samples >> DEBUG level output looks like this.                   
08:54:47 |  WARNING |  mylog.print_samples >> WARNING level output looks like this.                 
08:54:47 |    ERROR |  mylog.print_samples >> ERROR level output looks like this.                   
08:54:47 | CRITICAL |  mylog.print_samples >> CRITICAL level output looks like this.                
08:54:47 |     INFO |  mylog.print_samples >> Here is a very long message that probably will not fit
                                           >> on a single line. We will use it to demonstrate what  
                                           >> multi-line output looks like in each style.           
08:54:47 |     INFO |         mylog.status >> Some output from method "status" in class "SomeClass".
08:54:47 |     INFO | …very_very_long_name >> Output from a functio

08:54:47 mylog > Some output from method "status" in class "SomeClass".                             
08:54:47 mylog > Output from a function with a long name to demonstrate truncation or multiline,    
depending on the style.                                                                             

****************************************************************************************************


## Palettes

timbermafia includes a variety of colour palettes for each log level. This can be

useful in reducing log clutter in Streams, where the levelname would otherwise 

take up room

In [3]:
for my_palette in tm.palettes.palette_map:
    print(f'- Using palette: {my_palette}\n')
    # Clear the configured handlers each time, and disable fit_to_terminal in each style
    # because Jupyter notebooks don't work well with this feature.
    tm.basic_config(palette=my_palette, clear=True, silent=True, fit_to_terminal=False)
    demo()
    print()
    print('*'*100)

- Using palette: sensible

08:54:47 |     INFO |  mylog.print_samples >> INFO level output looks like this.                    
08:54:47 |    DEBUG |  mylog.print_samples >> DEBUG level output looks like this.                   
08:54:47 |  WARNING |  mylog.print_samples >> WARNING level output looks like this.                 
08:54:47 |    ERROR |  mylog.print_samples >> ERROR level output looks like this.                   
08:54:47 | CRITICAL |  mylog.print_samples >> CRITICAL level output looks like this.                
08:54:47 |     INFO |  mylog.print_samples >> Here is a very long message that probably will not fit
                                           >> on a single line. We will use it to demonstrate what  
                                           >> multi-line output looks like in each style.           
08:54:47 |     INFO |         mylog.status >> Some output from method "status" in class "SomeClass".
08:54:47 |     INFO | …very_very_long_name >> Output from a func

08:54:47 |     INFO | …very_very_long_name >> Output from a function with a long name to demonstrate
                                           >> truncation or multiline, depending on the style.      

****************************************************************************************************
- Using palette: ocean

08:54:47 |     INFO |  mylog.print_samples >> INFO level output looks like this.                    
08:54:47 |    DEBUG |  mylog.print_samples >> DEBUG level output looks like this.                   
08:54:47 |  WARNING |  mylog.print_samples >> WARNING level output looks like this.                 
08:54:47 |    ERROR |  mylog.print_samples >> ERROR level output looks like this.                   
08:54:47 | CRITICAL |  mylog.print_samples >> CRITICAL level output looks like this.                
08:54:47 |     INFO |  mylog.print_samples >> Here is a very long message that probably will not fit
                                           >> on a single line. We

As can be seen, the white or black texts enforced in the formats don't always go

well with each palette. For this reason you can supply your own format easily and

experiment with colours or formatting for each component, or remove them entirely.

## Creating a custom style

An instance Style configured with any preset can be generated and altered as follows.

In [4]:
my_style = tm.generate_default_style()  # my_style is now the default Style
my_style = tm.generate_style_from_preset('plain')  # my_style is now the plain Style
my_style.summarise()

- Current settings for style:
             format: {asctime} {name} > {message}
            datefmt: %H:%M:%S
            justify:
                      default: left
    padding_weights:
                      default: 1.0
                      message: 6.0

    truncate_fields: 
   truncation_chars: …
    fit_to_terminal: True
              width: 100
          max_width: 200
      column_escape: _
       clean_output: True
  colourised_levels: False
       short_levels: False
        description: A style emulating vanilla logging.
**************************************************


Now let's modify some style properties.

Let's imagine a threaded application where the thread name is of interest in the output.

This application is also OO-heavy, and it is useful to know from which class and 

method a given message is coming from.

The Style class has a number of tweakable properties to change how output looks.

In [5]:
# Let's change the format to include threads and function names.
# We'll underline the time to emphasise the start of a new record.
# We'll also make the message a bold, bright green just to see how it looks.
my_style.format = '{asctime:u} _ {levelname} _| {threadName} _| {name}.{funcName} _>> {message:b,>36}'

# Now let's change the justifiction so that the default is right, and have the message
# be left justified to create a visual focal point in the log output.
my_style.default_justification = 'right'  # 'l' or str.ljust work here too
my_style.set_justification('message', 'l')

# We want a tidy output, so we will truncate the funcName if it gets too long to display
# in the allotted width. Note that this also affects the log name, as they are in the same
# column.
my_style.truncate_field('funcName')

# We're working in Jupyter, so let's turn off the fit_to_terminal option.
my_style.fit_to_terminal = False

# Enable colourised output based on log levels
my_style.colourised_levels = True

# Now check our changes are registered.
my_style.summarise()

- Current settings for style:
             format: {asctime:u} _ {levelname} _| {threadName} _| {name}.{funcName} _>> {message:b,>36}
            datefmt: %H:%M:%S
            justify:
                      default: right
                      message: left
    padding_weights:
                      default: 1.0
                      message: 6.0

    truncate_fields: funcName
   truncation_chars: …
    fit_to_terminal: False
              width: 100
          max_width: 200
      column_escape: _
       clean_output: True
  colourised_levels: True
       short_levels: False
        description: A style emulating vanilla logging.
**************************************************


Our changes have been registered.

And let's configure it, and try it out with the synth palette.

In [6]:
tm.basic_config(style=my_style, palette='synth', silent=True, clear=True)
demo()

08:54:47      INFO | MainThre | …g.print_samples >> INFO level output looks like this.              
                           ad                                                                       
08:54:47     DEBUG | MainThre | …g.print_samples >> DEBUG level output looks like this.             
                           ad                                                                       
08:54:47   WARNING | MainThre | …g.print_samples >> WARNING level output looks like this.           
                           ad                                                                       
08:54:47     ERROR | MainThre | …g.print_samples >> ERROR level output looks like this.             
                           ad                                                                       
08:54:47  CRITICAL | MainThre | …g.print_samples >> CRITICAL level output looks like this.          
                           ad                                                              

In [7]:
my_style.set_weight('threadName', 1.3)
my_style.short_levels = True
my_style.width = 120

tm.basic_config(style=my_style, palette='synth', silent=True, clear=True)
demo()

08:54:47  I |     MainThread |    mylog.print_samples >> INFO level output looks like this.                             
08:54:47  D |     MainThread |    mylog.print_samples >> DEBUG level output looks like this.                            
08:54:47  W |     MainThread |    mylog.print_samples >> WARNING level output looks like this.                          
08:54:47  E |     MainThread |    mylog.print_samples >> ERROR level output looks like this.                            
08:54:47  C |     MainThread |    mylog.print_samples >> CRITICAL level output looks like this.                         
08:54:47  I |     MainThread |    mylog.print_samples >> Here is a very long message that probably will not fit on a    
                                                         single line. We will use it to demonstrate what multi-line     
                                                         output looks like in each style.                               
08:54:47  I |     MainThread |  

Much better!

Hopefully this shows that making your own styles and tweaking them to match your
specific needs is easy in timbermafia.

# Custom Palettes

Custom palettes can be configured in a similar way.

Let's generate one from the default palette, called "sensible"

In [8]:
my_palette = tm.generate_default_palette()
my_palette.summarise()

CRITICAL looks like this
ERROR looks like this
WARNING looks like this
INFO looks like this
DEBUG looks like this
NOTSET looks like this


Now let's change some colours, make something truly awful looking.

A list of all ANSI codes can be found in the top answer at the following StackExchange question:

https://stackoverflow.com/questions/4842424/list-of-ansi-color-escape-sequences

Note that not all terminal emulators, including the output section of Jupyter notebooks, support all
ANSI format codes, but usually support 8-bit colour codes.

In [9]:
my_palette.set_level(logging.INFO, fg=201, bg=190, codes=1)  # Bold pink on yelow
my_palette.set_level(logging.DEBUG, fg=46, bg=17, codes=4)  # Underlined green on dark blue
my_palette.set_level(logging.CRITICAL, fg=213, bg=54, codes=2) # Normal lilac on dark purple
my_palette.set_level(logging.WARNING, fg=172, bg=195) # Beige on baby blue
my_palette.summarise()

CRITICAL looks like this
ERROR looks like this
WARNING looks like this
INFO looks like this
DEBUG looks like this
NOTSET looks like this


This looks suirably horrendous, let's use it with our demo function.

We'll also use a format with no custom colours for log record components, and

expand the date/time format to include the day of the year.

In [10]:
my_fmt = '{asctime} _| {name} _>> {message}'
my_datefmt = '%Y:%m:%d %H%M%S'
tm.basic_config(palette=my_palette, format=my_fmt, datefmt=my_datefmt,
                silent=True, clear=True)
demo()

2020:05:09 085447 |       mylog >> INFO level output looks like this.                               
2020:05:09 085447 |       mylog >> DEBUG level output looks like this.                              
2020:05:09 085447 |       mylog >> WARNING level output looks like this.                            
2020:05:09 085447 |       mylog >> ERROR level output looks like this.                              
2020:05:09 085447 |       mylog >> CRITICAL level output looks like this.                           
2020:05:09 085447 |       mylog >> Here is a very long message that probably will not fit on a      
                                   single line. We will use it to demonstrate what multi-line output
                                   looks like in each style.                                        
2020:05:09 085447 |       mylog >> Some output from method "status" in class "SomeClass".           
2020:05:09 085447 |       mylog >> Output from a function with a long name to demonstrate t

Truly hideous, but this should demonstrate the flexibility in timbermafia's palettes.

If you make one that is particularly eye-catching, or caters to a particular type of

colour-blindness especially well, please get in touch with me and it might become

a preset palette.